In [1]:
!pip install openai-whisper groq piper-tts gradio
!apt install ffmpeg  # Needed for Whisper STT


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.3 MB/s eta 0:0

In [4]:
!wget -O en_US.onnx https://huggingface.co/rhasspy/piper-voices/resolve/main/en_US/en_US-libritts-high.onnx
!wget -O en_US.json https://huggingface.co/rhasspy/piper-voices/resolve/main/en_US/en_US-libritts-high.onnx.json


--2025-02-23 15:15:04--  https://huggingface.co/rhasspy/piper-voices/resolve/main/en_US/en_US-libritts-high.onnx
Resolving huggingface.co (huggingface.co)... 18.244.202.68, 18.244.202.60, 18.244.202.118, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.68|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-02-23 15:15:04 ERROR 404: Not Found.

--2025-02-23 15:15:04--  https://huggingface.co/rhasspy/piper-voices/resolve/main/en_US/en_US-libritts-high.onnx.json
Resolving huggingface.co (huggingface.co)... 18.244.202.68, 18.244.202.60, 18.244.202.118, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.68|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-02-23 15:15:04 ERROR 404: Not Found.



In [13]:
from piper import PiperVoice

# Load Piper model with voice file & config
voice = PiperVoice("en_US.onnx", "en_US.json")


In [14]:
import whisper
from piper import PiperVoice

# Load Whisper model for Speech-to-Text
whisper_model = whisper.load_model("base")

# Load Piper model (Ensure voice files are downloaded first)
voice = PiperVoice("en_US.onnx", "en_US.json")


In [15]:
def speech_to_text(audio_file):
    result = whisper_model.transcribe(audio_file)
    return result["text"]


In [16]:
def get_ai_response(query):
    url = "https://api.groq.com/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": GROQ_MODEL,
        "messages": [{"role": "user", "content": query}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"]


In [17]:
def text_to_speech(text):
    output_audio = "response.wav"
    voice.synthesize(text, output_audio)
    return output_audio


In [19]:
def chatbot(audio_file):
    user_query = speech_to_text(audio_file)
    print("User Query:", user_query)  # Debugging output

    ai_response = get_ai_response(user_query)
    print("AI Response:", ai_response)  # Debugging output

    audio_output = text_to_speech(ai_response)

    return ai_response, audio_output


takes input text and audio

In [1]:
!pip install gradio torch openai-whisper groq piper-tts soundfile

import gradio as gr
import whisper
import torch
import os
import time
from groq import Groq
from piper import PiperVoice # Corrected import statement
import soundfile as sf

# ✅ Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ Load Whisper for Speech-to-Text
whisper_model = whisper.load_model("base").to(device)

# ✅ Load Piper for Text-to-Speech
piper_model_path = "en_US.onnx" # Corrected path to piper model
piper_config_path = "en_US.json" # Corrected path to piper config
voice = PiperVoice(piper_model_path, piper_config_path)

# ✅ Set Up Groq API
groq_client = Groq(api_key="groqapikey")

def chatbot(audio_file=None, text_input=None):
    """Process text or voice input and return AI-generated response as text and speech."""

    if audio_file:  # 🎤 If user speaks
        text = whisper_model.transcribe(audio_file)["text"]
    elif text_input:  # ⌨️ If user types
        text = text_input
    else:
        return "Please provide text or voice input.", None

    # ⏳ Track API Time
    start_time = time.time()
    response = groq_client.generate(text)
    end_time = time.time()
    print(f"API Response Time: {end_time - start_time} sec")  # Debug speed

    # 🔊 Convert Text Response to Speech
    response_audio_path = "response.wav"
    voice.synthesize(response, response_audio_path, config={"length_scale": 1.2})  # Faster TTS

    return response, response_audio_path

# ✅ Updated Interface with Both Text & Audio Inputs
iface = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Audio(type="filepath", label="🎤 Speak your question (Optional)"),
        gr.Textbox(label="⌨️ Or type your question")
    ],
    outputs=[
        gr.Textbox(label="📝 AI Response"),
        gr.Audio(type="filepath", label="🔊 Hear the response")
    ],
    title="AI Tourism Guide",
    description="Speak or type your tourism-related question and get AI-powered voice responses.",
    allow_flagging="never"  # Remove "Flag" button
)

iface.launch(share=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 128MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b51541bb81fc6390ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
